In [1]:
import pandas as pd

#get pandas and sql to work together
import psycopg2 as pg
import pandas.io.sql as pd_sql

from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import matplotlib.pyplot as plt

In [2]:
params = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'user': 'agar',
    'dbname': 'votes',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**params)

In [17]:
sql_query_demographics = """
select 
"Code" as district_id, 
cast("2010 Census Adult Population by Race" as float) as white_adult_percent, 
cast("Unnamed: 33" as float) as black_adult_percent, 
cast("Unnamed: 34" as float) as latino_adult_percent,
cast("Unnamed: 35" as float) as asian_adult_percent,
cast("Unnamed: 36" as float) as native_adult_percent,
cast("Unnamed: 37" as float) as other_adult_percent,
cast("2016 American Community Survey Income and Education" as float) as bachelors,
cast("Unnamed: 40" as float) as white_bachelors,
cast("Non-College White Share" as float) as white_no_college,
cast (REPLACE(replace("Unnamed: 42", ',', ''), '$', '') as integer) as income_median,
cast(replace("Census Voting Age Population_Includes Latinos of Any Race", ',', '') as integer) as total_vote,
cast(replace("Unnamed: 94", ',', '') as integer) as white_vote,
cast(replace("Unnamed: 95", ',', '') as integer) as black_vote,
cast(replace("Unnamed: 96", ',', '') as integer) as latino_vote,
cast(replace("Unnamed: 97", ',', '') as integer) as asian_vote,
cast(replace("Unnamed: 98", ',', '') as integer) as native_vote,
cast(replace("Unnamed: 99", ',', '') as integer) as other_vote,
cast(replace("Census Total Population_Includes Latinos of Any Race", ',', '') as integer) as total_pop,
cast(replace("Unnamed: 87", ',', '') as integer) as white_pop,
cast(replace("Unnamed: 88", ',', '') as integer) as black_pop,
cast(replace("Unnamed: 89", ',', '') as integer) as latino_pop,
cast(replace("Unnamed: 90", ',', '') as integer) as asian_pop,
cast(replace("Unnamed: 91", ',', '') as integer) as native_pop,
cast(replace("Unnamed: 92", ',', '') as integer) as other_pop

from "demographic_115_con" 
"""

In [18]:
sql_query_largest_geo = """
select 
"District" as district_id,
"p_of CD" as larg1_percent_pop,
"p_of CD.1" as larg2_percent_pop,
"p_of CD.2" as larg3_percent_pop
from "geography_116+" g 
"""

In [37]:
sql_query_metro = """
select 
"District" as district_id,
"p_of CD" as metro1_percent_pop,
"p_of CD.1" as metro2_percent_pop,
"p_of CD.2" as metro3_percent_pop,
"p_no metro" as metro_none_percent_pop
from "geography_116_metro"  
"""

In [20]:
sql_query_votes = """
select year,  
case 
	when district<10 
		then concat(state_po, '-0', district)
	when district>9	
		then concat(state_po, '-', district)
		end as district_id,
state_fips, party,
cast(replace(candidatevotes, ',', '') as integer) as candidate_votes, 
totalvotes

from "H_of_Rep" hor 

where year>2009 and year<2020
order by year, district_id, party
"""

In [21]:
df_demographics = pd.read_sql_query(sql_query_demographics, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [48]:
df_demographics.head(440)

,district_id,white_adult_percent,black_adult_percent,latino_adult_percent,asian_adult_percent,native_adult_percent,other_adult_percent,bachelors,white_bachelors,white_no_college,...,asian_vote,native_vote,other_vote,total_pop,white_pop,black_pop,latino_pop,asian_pop,native_pop,other_pop
0,USA,67.0,11.7,14.2,5.0,0.7,1.6,31.2,35.0,45.3,...,11637156,1607263,3704719,308742908,196816822,37817343,50476704,14950101,2259941,6749678
1,AL-01,68.5,25.6,2.5,1.3,1.1,1.0,24.0,27.7,49.6,...,6918,5524,4994,682423,449539,187508,19086,9296,7644,9350
2,AL-02,66.5,27.8,3.0,1.2,0.5,1.0,21.8,24.2,50.1,...,6059,2371,5072,686078,438782,201723,24648,7900,3072,9953
3,AL-03,71.5,23.9,2.3,1.1,0.3,0.9,22.8,23.7,54.4,...,5992,1705,4681,680857,473473,170355,17932,7923,2117,9057
4,AL-04,86.9,6.5,4.3,0.5,0.7,1.0,17.0,17.8,73.0,...,2369,3870,5455,681487,578453,46163,38837,3200,5218,9616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,WI-05,91.7,1.4,3.9,2.1,0.3,0.7,36.9,36.9,58.3,...,11315,1626,3765,711328,636934,11589,35609,16161,2234,8801
432,WI-06,93.0,1.5,2.8,1.7,0.4,0.6,26.4,26.8,68.5,...,9099,2414,3188,711209,647963,10776,27092,14832,3044,7502
433,WI-07,94.5,0.5,1.3,1.2,1.7,0.8,23.1,23.4,72.5,...,6322,9538,4392,711417,661125,4157,12549,10541,13985,9060
434,WI-08,91.3,0.9,3.2,1.7,2.1,0.8,25.4,26.0,68.3,...,8908,11474,4103,710441,631736,7895,30688,14006,16564,9552


In [23]:
df_largest_geo = pd.read_sql_query(sql_query_largest_geo, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [24]:
df_largest_geo.head()

,district_id,larg1_percent_pop,larg2_percent_pop,larg3_percent_pop
0,AK-AL,41.1,4.4,4.4
1,AL-01,28.6,3.3,3.2
2,AL-02,19.7,9.6,5.0
3,AL-03,7.8,4.8,3.9
4,AL-04,5.4,3.1,2.2


In [38]:
df_metro = pd.read_sql_query(sql_query_metro, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [46]:
df_metro.head()

,district_id,metro1_percent_pop,metro2_percent_pop,metro3_percent_pop,metro_none_percent_pop
0,AK-00,53.6,13.7,4.4,26.3
1,AL-01,60.5,26.7,5.6,7.2
2,AL-02,41.0,21.3,7.4,14.2
3,AL-03,20.5,17.4,12.2,14.3
4,AL-04,15.3,13.6,11.8,17.7


In [45]:
df_metro["district_id"][0] = "AK-00"

/Users/agar/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
district_merge_1 = pd.merge(df_metro, df_largest_geo, how='inner', on="district_id")

In [76]:
district_merge_1b = pd.merge(district_merge_1, df_demographics, how='inner', on="district_id")

In [78]:
district_merge_1b.head()

,district_id,metro1_percent_pop,metro2_percent_pop,metro3_percent_pop,metro_none_percent_pop,larg1_percent_pop,larg2_percent_pop,larg3_percent_pop,white_adult_percent,black_adult_percent,...,asian_vote,native_vote,other_vote,total_pop,white_pop,black_pop,latino_pop,asian_pop,native_pop,other_pop
0,AK-AL,53.6,13.7,4.4,26.3,41.1,4.4,4.4,68.4,3.1,...,32158,69054,23471,708570,454831,21862,39038,44357,102124,46358
1,AL-01,60.5,26.7,5.6,7.2,28.6,3.3,3.2,68.5,25.6,...,6918,5524,4994,682423,449539,187508,19086,9296,7644,9350
2,AL-02,41.0,21.3,7.4,14.2,19.7,9.6,5.0,66.5,27.8,...,6059,2371,5072,686078,438782,201723,24648,7900,3072,9953
3,AL-03,20.5,17.4,12.2,14.3,7.8,4.8,3.9,71.5,23.9,...,5992,1705,4681,680857,473473,170355,17932,7923,2117,9057
4,AL-04,15.3,13.6,11.8,17.7,5.4,3.1,2.2,86.9,6.5,...,2369,3870,5455,681487,578453,46163,38837,3200,5218,9616


In [50]:
new = district_merge_1["district_id"].str.split("-", n = 1, expand = True) 

In [54]:
new.head()

,0,1
0,AK,AL
1,AL,01
2,AL,02
3,AL,03
4,AL,04


In [55]:
new_num = new[1].replace("AL", '00')

In [56]:
new_num.head()

0    00
1    01
2    02
3    03
4    04
Name: 1, dtype: object

In [63]:
news = pd.concat([new[0], new_num], axis=1, sort=False)

In [61]:
news_district_fix = news[0] + "-" + news[1]

In [62]:
news_district_fix

0      AK-00
1      AL-01
2      AL-02
3      AL-03
4      AL-04
       ...  
430    WI-08
431    WV-01
432    WV-02
433    WV-03
434    WY-00
Length: 435, dtype: object

In [79]:
district_merge_2 = pd.concat([news_district_fix, district_merge_1b], axis=1, sort=False)

In [80]:
district_merge_2 = district_merge_2.drop(columns = "district_id")

In [81]:
district_merge_2.columns

Index([                       0,     'metro1_percent_pop',
           'metro2_percent_pop',     'metro3_percent_pop',
       'metro_none_percent_pop',      'larg1_percent_pop',
            'larg2_percent_pop',      'larg3_percent_pop',
          'white_adult_percent',    'black_adult_percent',
         'latino_adult_percent',    'asian_adult_percent',
         'native_adult_percent',    'other_adult_percent',
                    'bachelors',        'white_bachelors',
             'white_no_college',          'income_median',
                   'total_vote',             'white_vote',
                   'black_vote',            'latino_vote',
                   'asian_vote',            'native_vote',
                   'other_vote',              'total_pop',
                    'white_pop',              'black_pop',
                   'latino_pop',              'asian_pop',
                   'native_pop',              'other_pop'],
      dtype='object')

In [82]:
district_merge_2 = district_merge_2.rename(columns={0: "district_id"})

In [83]:
district_merge_2.head()

,district_id,metro1_percent_pop,metro2_percent_pop,metro3_percent_pop,metro_none_percent_pop,larg1_percent_pop,larg2_percent_pop,larg3_percent_pop,white_adult_percent,black_adult_percent,...,asian_vote,native_vote,other_vote,total_pop,white_pop,black_pop,latino_pop,asian_pop,native_pop,other_pop
0,AK-00,53.6,13.7,4.4,26.3,41.1,4.4,4.4,68.4,3.1,...,32158,69054,23471,708570,454831,21862,39038,44357,102124,46358
1,AL-01,60.5,26.7,5.6,7.2,28.6,3.3,3.2,68.5,25.6,...,6918,5524,4994,682423,449539,187508,19086,9296,7644,9350
2,AL-02,41.0,21.3,7.4,14.2,19.7,9.6,5.0,66.5,27.8,...,6059,2371,5072,686078,438782,201723,24648,7900,3072,9953
3,AL-03,20.5,17.4,12.2,14.3,7.8,4.8,3.9,71.5,23.9,...,5992,1705,4681,680857,473473,170355,17932,7923,2117,9057
4,AL-04,15.3,13.6,11.8,17.7,5.4,3.1,2.2,86.9,6.5,...,2369,3870,5455,681487,578453,46163,38837,3200,5218,9616


In [27]:
df_votes = pd.read_sql_query(sql_query_votes, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [28]:
df_votes.head()

,year,district_id,state_fips,party,candidate_votes,totalvotes
0,2010,AK-00,2,democrat,77606,254335.0
1,2010,AK-00,2,republican,175384,254335.0
2,2010,AK-00,2,None,1345,254335.0
3,2010,AL-01,1,constitution,26357,156281.0
4,2010,AL-01,1,republican,129063,156281.0


In [85]:
df_votes.tail()

,year,district_id,state_fips,party,candidate_votes,totalvotes
7078,2018,WY-00,56,constitution,6070,201245.0
7079,2018,WY-00,56,democrat,59903,201245.0
7080,2018,WY-00,56,libertarian,6918,201245.0
7081,2018,WY-00,56,republican,127963,201245.0
7082,2018,WY-00,56,None,391,201245.0


In [87]:
sql_query_winners_ten = """
select year, party, district_id
from "Winners+" w 
where year>2009 and year<2020
order by year, district_id, party
"""

In [88]:
df_winners_ten = pd.read_sql_query(sql_query_winners_ten, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [89]:
df_winners_ten

,year,party,district_id
0,2010,republican,AK-00
1,2010,republican,AL-01
2,2010,republican,AL-02
3,2010,republican,AL-03
4,2010,republican,AL-04
...,...,...,...
2170,2018,republican,WI-08
2171,2018,republican,WV-01
2172,2018,republican,WV-02
2173,2018,republican,WV-03


In [94]:
winner_votes = pd.merge(df_winners_ten, df_votes, how='left', on=["year", "district_id", "party"])

In [91]:
sql_query_flips_agg = """
select * from
(
select *, 
sum(party_change_simple) over (partition by district_id order by year asc) as sum_flips_total
from "Flips" 
) as total_flips
where year>2009 and year<2019 
"""

In [92]:
df_flips_agg = pd.read_sql_query(sql_query_flips_agg, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [93]:
df_flips_agg.head()

,year,district_id,party_change_simple,sum_flips_total
0,2010,AK-00,0.0,0.0
1,2012,AK-00,0.0,0.0
2,2014,AK-00,0.0,0.0
3,2016,AK-00,0.0,0.0
4,2018,AK-00,0.0,0.0


In [95]:
winner_votes_agg = pd.merge(df_flips_agg, winner_votes, how='left', on=["year", "district_id"])

In [96]:
winner_votes_agg.head()

,year,district_id,party_change_simple,sum_flips_total,party,state_fips,candidate_votes,totalvotes
0,2010,AK-00,0.0,0.0,republican,2,175384,254335.0
1,2012,AK-00,0.0,0.0,republican,2,185296,289804.0
2,2014,AK-00,0.0,0.0,republican,2,142572,279741.0
3,2016,AK-00,0.0,0.0,republican,2,155088,308198.0
4,2018,AK-00,0.0,0.0,republican,2,149779,282166.0


In [98]:
complete_df = pd.merge(winner_votes_agg, district_merge_2, how='left', on=["district_id"])

In [99]:
complete_df.head()

,year,district_id,party_change_simple,sum_flips_total,party,state_fips,candidate_votes,totalvotes,metro1_percent_pop,metro2_percent_pop,...,asian_vote,native_vote,other_vote,total_pop,white_pop,black_pop,latino_pop,asian_pop,native_pop,other_pop
0,2010,AK-00,0.0,0.0,republican,2,175384,254335.0,53.6,13.7,...,32158.0,69054.0,23471.0,708570.0,454831.0,21862.0,39038.0,44357.0,102124.0,46358.0
1,2012,AK-00,0.0,0.0,republican,2,185296,289804.0,53.6,13.7,...,32158.0,69054.0,23471.0,708570.0,454831.0,21862.0,39038.0,44357.0,102124.0,46358.0
2,2014,AK-00,0.0,0.0,republican,2,142572,279741.0,53.6,13.7,...,32158.0,69054.0,23471.0,708570.0,454831.0,21862.0,39038.0,44357.0,102124.0,46358.0
3,2016,AK-00,0.0,0.0,republican,2,155088,308198.0,53.6,13.7,...,32158.0,69054.0,23471.0,708570.0,454831.0,21862.0,39038.0,44357.0,102124.0,46358.0
4,2018,AK-00,0.0,0.0,republican,2,149779,282166.0,53.6,13.7,...,32158.0,69054.0,23471.0,708570.0,454831.0,21862.0,39038.0,44357.0,102124.0,46358.0


In [104]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2228
Data columns (total 39 columns):
year                      2229 non-null int64
district_id               2229 non-null object
party_change_simple       2229 non-null float64
sum_flips_total           2229 non-null float64
party                     2229 non-null object
state_fips                2229 non-null int64
candidate_votes           2229 non-null int64
totalvotes                2229 non-null float64
metro1_percent_pop        2229 non-null float64
metro2_percent_pop        2229 non-null float64
metro3_percent_pop        2229 non-null float64
metro_none_percent_pop    2229 non-null float64
larg1_percent_pop         2229 non-null float64
larg2_percent_pop         2229 non-null float64
larg3_percent_pop         2229 non-null float64
white_adult_percent       2229 non-null float64
black_adult_percent       2229 non-null float64
latino_adult_percent      2229 non-null float64
asian_adult_percent       2229 non-nu

In [103]:
complete_df = complete_df.fillna(0)

In [105]:
complete_df.to_csv('/Users/agar/_METIS/exercises/Project_3/data_source/complete_df') 

In [4]:
from sqlalchemy import create_engine
connection_string = f'postgres://agar:{params["host"]}@{params["host"]}:{params["port"]}/votes'
engine = create_engine(connection_string, pool_pre_ping=True)

In [107]:
complete_df.iloc[:0].to_sql("Complete", engine, index=False)
complete_df.iloc[:].to_sql("Complete", engine, index=False, if_exists = 'append', chunksize = 1000)

In [7]:
 sql_query_finance = """
select * from "financials"
"""

In [8]:
df_financials = pd.read_sql_query(sql_query_finance, connection, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

In [32]:
#get only House of Representatives
mask = df_financials['Cand_Office'] == "H"
house_df = df_financials[mask]

#select financial columns 
df_in = house_df[['Cand_State', 'Cand_Office_Dist','year', 'Total_Receipt', 'Total_Disbursement', 'Individual_Itemized_Contribution',
       'Individual_Unitemized_Contribution', 'Individual_Contribution',
       'Other_Committee_Contribution', 'Party_Committee_Contribution',
       'Cand_Contribution', 'Total_Contribution', 'Operating_Expenditure']]

#sum expenditures by district and year
df_financial_sum = df_in.groupby(['Cand_State', 'Cand_Office_Dist','year']).agg('sum')
df_financial_sum_r = df_financial_sum.reset_index()

#set columns for joins
df_financial_sum_r['Cand_Office_Dist'] = df_financial_sum_r.Cand_Office_Dist.map("{:02}".format)
df_financial_sum_r['ID_DIST'] =  df_financial_sum_r['year'].astype(str)+ "-" + df_financial_sum_r['Cand_State'] + "-" + df_financial_sum_r['Cand_Office_Dist'].astype(str) 
df_financial_sum_r['district_id'] =  df_financial_sum_r['Cand_State']+ "-" + df_financial_sum_r['Cand_Office_Dist'].astype(str) 
df_r.to_csv("financials_df.csv")

df_r.to_csv("financials_df.csv")